Importing Libraries

In [1]:
import cv2
import face_recognition as fr
import numpy as np

# Initialize video capture (0 for webcam, or provide a video file path)
video_capture = cv2.VideoCapture(0)

# Load and encode known face(s)
known_face_encodings = []
known_face_names = []

# Load known faces properly
image_path = "images/boy.jpg"  # Use forward slashes for cross-platform compatibility
try:
    known_image = fr.load_image_file(image_path)  # Load image
    known_encoding = fr.face_encodings(known_image)

    if known_encoding:  # Ensure the face is detected
        known_face_encodings.append(known_encoding[0])
        known_face_names.append("Saurav")
    else:
        print(f"Warning: No face found in {image_path}")

except Exception as e:
    print(f"Error loading {image_path}: {e}")

if not video_capture.isOpened():
    print("Error: Could not open video capture.")
    exit()

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to grab frame")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    face_locations = fr.face_locations(rgb_frame)
    face_encodings = fr.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        name = "Unknown"

        if known_face_encodings:  # Ensure known encodings exist
            matches = fr.compare_faces(known_face_encodings, face_encoding)
            face_distances = fr.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)

            if matches[best_match_index]:  # Match found
                name = known_face_names[best_match_index]

        # Draw rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)

        # Display name
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 